In [3]:
#SET UP
#import sys
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install matplotlib

/usr/local/codem/public_use_anaconda/bin/python: can't open file 'python': [Errno 2] No such file or directory


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from survival import DecayMIRModel
from survival import MIRModel

ImportError: No module named src.survival

In [4]:
#PROCESS INPUTS
df = pd.read_csv('/home/j/temp/fed1/SEER_case_analysis_formatted_add_unknown_to_other_mortality_all_sites_each_age_params_n=10_slope_off10_4_20_21.csv')

#remove NAs and MIR >1
df = df.dropna()
df = df[df['mi_ratio']<=1]
#df = df[df['slope']<0]

df.head()

,Unnamed: 0,age_name,age_group_id,V1,cause_name.x,cases,deaths,mi_ratio,five_year_cs_survival,five_year_observed_survival,...,sex_id,acause,disease_period,age,cause_name.y,sex,slope,slope_SE,slope_upper,slope_lower
0,1,00-04 years,1,1,Acute Lymphocytic Leukemia,6860,327.0,0.047668,0.932799,0.924490,...,3,Acute Lymphocytic Leukemia,10,0,All Sites,3,-0.377813,0.030319,-0.317174,-0.438452
1,2,00-04 years,1,2,Acute Monocytic Leukemia,167,52.0,0.311377,0.676647,0.634731,...,3,Acute Monocytic Leukemia,10,0,All Sites,3,-0.377813,0.030319,-0.317174,-0.438452
2,3,00-04 years,1,3,Acute Myeloid Leukemia,1092,270.0,0.247253,0.729853,0.691392,...,3,Acute Myeloid Leukemia,10,0,All Sites,3,-0.377813,0.030319,-0.317174,-0.438452
3,4,00-04 years,1,4,"Aleukemic, Subleukemic and NOS",56,5.0,0.089286,0.910714,0.821429,...,3,"Aleukemic, Subleukemic and NOS",10,0,All Sites,3,-0.377813,0.030319,-0.317174,-0.438452
4,5,00-04 years,1,5,Appendix,3,0.0,0.000000,1.000000,1.000000,...,3,Appendix,10,0,All Sites,3,-0.377813,0.030319,-0.317174,-0.438452


In [ ]:
df[df['slope']>0].head()

In [5]:
#RUN MODEL
num_years = 5

model = DecayMIRModel(df['mi_ratio'],
                 df['other_mortality'],
                 df['disease_period'],
                 df['slope'])
model.compute_base_excess_mortality()
survival_rate = model.get_survival_rate(num_years=num_years)

df['base_excess_mortality'] = model.base_excess_mortality
df['abs_survival_rate'] = survival_rate['abs']
df['rel_survival_rate'] = survival_rate['rel']

In [6]:
#RUN OLD MODEL TO COMPARE
num_years = 5

model = MIRModel(df['mi_ratio'],
                 df['other_mortality'], disease_period=10)
model.compute_excess_mortality()
survival_rate = model.get_survival_rate(num_years=num_years)

df['excess_mortality_old'] = model.excess_mortality
df['abs_survival_rate_old'] = survival_rate['abs']
df['rel_survival_rate_old'] = survival_rate['rel']

In [7]:
df.head()

,Unnamed: 0,age_name,age_group_id,V1,cause_name.x,cases,deaths,mi_ratio,five_year_cs_survival,five_year_observed_survival,...,slope,slope_SE,slope_upper,slope_lower,base_excess_mortality,abs_survival_rate,rel_survival_rate,excess_mortality_old,abs_survival_rate_old,rel_survival_rate_old
0,1,00-04 years,1,1,Acute Lymphocytic Leukemia,6860,327.0,0.047668,0.932799,0.924490,...,-0.377813,0.030319,-0.317174,-0.438452,0.023217,0.949300,0.961132,0.004941,0.960624,0.975464
1,2,00-04 years,1,2,Acute Monocytic Leukemia,167,52.0,0.311377,0.676647,0.634731,...,-0.377813,0.030319,-0.317174,-0.438452,0.179243,0.654426,0.727016,0.041425,0.713151,0.805056
2,3,00-04 years,1,3,Acute Myeloid Leukemia,1092,270.0,0.247253,0.729853,0.691392,...,-0.377813,0.030319,-0.317174,-0.438452,0.137239,0.711874,0.785568,0.031378,0.756538,0.849419
3,4,00-04 years,1,4,"Aleukemic, Subleukemic and NOS",56,5.0,0.089286,0.910714,0.821429,...,-0.377813,0.030319,-0.317174,-0.438452,0.051769,0.623755,0.914812,0.013885,0.580161,0.926056
4,5,00-04 years,1,5,Appendix,3,0.0,0.000000,1.000000,1.000000,...,-0.377813,0.030319,-0.317174,-0.438452,0.000000,0.999960,1.000000,0.000000,0.999950,1.000000


In [8]:
df.to_csv('/home/j/temp/fed1/testing_new_decay_survival_model_outputs_on_case_analysis_data_all_cancer_specific_age_parameters_n=10_slope_off10_with_old_method.csv')